Since our data vaccination data is somewhat incomplete lets see if we can boost the model with some sythetic data. 

In [1]:
from pycaret.regression import *

# importing the required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import folium 
from folium import plugins

# Disable warnings 
import warnings
warnings.filterwarnings('ignore')

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf . Check the section C-API 
incompatibility at the Troubleshooting ImportError section at 
https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to 
solve this problem .

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/27/vtcvlrx512ncx4ngbtrg5jz40000gn/T/ipykernel_14759/2965032269.py:1 in <cell line:  │
│ 1>                                                                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/27/vtcvlrx512ncx4ngbtrg5jz40000gn/T/ipykernel_14759/2965032269.py'                 │
│                                                                                                  │
│ /usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/pycaret/__init_ │
│ _.py:1 in <module>                                                                               │
│                                                                                                  │
│ ❱ 1 from pycaret.utils import __version__                                                        │
│   2 from pycaret.utils._show_versions import show_versions                                       │
│   3                                                                                              │
│   4 __all__ = ["show_versions", "__version__"]                                                   │
│                                                                                                  │
│ /usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/pycaret/utils/_ │
│ _init__.py:1 in <module>                                                                         │
│                                                                                                  │
│ ❱  1 from sklearn.metrics._scorer import _PredictScorer, get_scorer  # type: ignore              │
│    2                                                                                             │
│    3 from pycaret.utils.generic import check_metric                                              │
│    4                                                                                             │
│                                                                                                  │
│ /usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/sklearn/__init_ │
│ _.py:82 in <module>                                                                              │
│                                                                                                  │
│    79 │   # it and importing it first would fail if the OpenMP dll cannot be found.              │
│    80 │   from . import _distributor_init  # noqa: F401                                          │
│    81 │   from . import __check_build  # noqa: F401                                              │
│ ❱  82 │   from .base import clone                                                                │
│    83 │   from .utils._show_versions import show_versions                                        │
│    84 │                                                                                          │
│    85 │   __all__ = [                                                                            │
│                                                                                                  │
│ /usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/sklearn/base.py │
│ :17 in <module>                                                                                  │
│                                                                                                  │
│     14                                                                                           │
│     15 from . import __version__                                                                 │
│     16 from ._config import get_config                     

In [ ]:
# load dataframe for feature analysis
combined_data = catalog.load("merge_who_ears_owid_data")
combined_data['date'] = pd.to_datetime(combined_data['date'], format = '%Y-%m-%d')
# combined_data["date"] = pd.to_datetime(combined_data["date"])


In [ ]:
features = combined_data[['date', 'mis_and_disinformation', 'mis_and_disinformation_complaints', 
                          'mis_and_disinformation_delta', 'mis_and_disinformation_male', 
                          'mis_and_disinformation_percent', 'mis_and_disinformation_questions', 
                          'mis_and_disinformation_female',
                         'myths', 'myths_complaints', 'myths_delta', 'myths_female', 
                          'myths_male', 'myths_percent', 'myths_questions','new_vaccinations']]

features = features.dropna()

In [ ]:
# lets setup our first session for eval
# we chose total cases since its a complete set AND is has high correlation with the other features (Pearson)
session_2 = setup(features, target = 'new_vaccinations', 
                  session_id=12, 
                  log_experiment=False, 
                  experiment_name='new_vaccinations_1')

In [ ]:
# We can now compare all the regression models available 
best_model = compare_models()

And now for some synthetic data creation with dGAN https://github.com/gretelai/gretel-synthetics/blob/master/examples/timeseries_dgan.ipynb

In [ ]:
#Uncomment to install ydata-synthetic lib
!pip install gretel-synthetics --upgrade
!pip install torch numpy pandas matplotlib scikit-learn

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.dates as md

import torch

from gretel_synthetics.timeseries_dgan.dgan import DGAN
from gretel_synthetics.timeseries_dgan.config import DGANConfig, OutputType


In [ ]:
# Plot the 4 columns over the 4+ months of data
for c in features.columns:
    if c in ['myths', 'mis_and_disinformation', 'new_vaccinations']:
        plt.plot(features["date"], features[c], label=c)
plt.xticks(rotation=90)
plt.legend()
plt.ylabel("Feature Values")
plt.xlabel("Date")
plt.show()

In [ ]:
print(features.shape)
# DGAN needs many example time series to train. Split into 1-day slices to
# create multiple examples.
features = features.drop(columns="date").to_numpy()
# Obsevations every 10 minutes, so 144 * 10 minutes = 1 day
n = features.shape[0]
features = features[:(n),:].reshape(-1, 1, features.shape[1])
# Shape is now (# examples, # time points, # features)
print(features.shape)

In [ ]:
features.shape

In [ ]:

# Train DGAN model
model = DGAN(DGANConfig(
    max_sequence_len=features.shape[1],
    sample_len=1,
    batch_size=min(1000, features.shape[0]),
    apply_feature_scaling=True,
    apply_example_scaling=False,
    use_attribute_discriminator=False,
    generator_learning_rate=1e-4,
    discriminator_learning_rate=1e-4,
    epochs=10000,
))

model.train_numpy(
    features,
    feature_types=[OutputType.CONTINUOUS] * features.shape[2],
)

# Generate synthetic data
_, synthetic_features = model.generate_numpy(20000)


In [ ]:
_, synthetic_features = model.generate_numpy(20000)



In [ ]:
# Compare distribution of T_out values
plt.hist([features[:,:,3].flatten(), synthetic_features[:,:,3].flatten()], 
         label=["real", "synthetic"],
         bins=25,
         density=True)
plt.legend()
plt.xlabel("Values")
plt.ylabel("Density")
plt.show()

In [ ]:
# Compare distribution of 1-step (10 minute) diffs for T_out
real_diffs = np.diff(features, axis=1)
synthetic_diffs = np.diff(synthetic_features, axis=1)

plt.hist([real_diffs[:,:,3].flatten(), synthetic_diffs[:,:,3].flatten()],
         label=["real", "synthetic"],
         bins=25,
         density=True)
plt.legend()
plt.xlabel("value change")
plt.ylabel("Density")
plt.show()


In [ ]:
# pd.DataFrame(synthetic_features).to_csv('sample.csv')
model.save("synthetic_model")


In [ ]:
# Lets make a return to the 2-d world by squeezeing our 3d out 
df = pd.DataFrame(np.squeeze(synthetic_features))



In [ ]:
df.to_csv('sample.csv')

In [ ]:
features.to_csv('original_set.csv')

In [ ]:
import datetime
# Add a column of numbers to the dataframe
df['numbers'] = range(20000)

# Get today's date
today = datetime.datetime.today()

# Generate a range of dates starting from today and ending today+9 days.
date_list = pd.date_range(today, periods=20000)

# Add date column to the dataframe
df = df.assign(date=date_list)

In [ ]:
dtale.show(df)

In [ ]:
df.columns

In [ ]:
col_names = ['mis_and_disinformation', 'mis_and_disinformation_complaints', 
 'mis_and_disinformation_delta', 'mis_and_disinformation_male', 
 'mis_and_disinformation_percent', 'mis_and_disinformation_questions', 
 'mis_and_disinformation_female',
                         'myths', 'myths_complaints', 'myths_delta', 'myths_female', 
 'myths_male', 'myths_percent', 'myths_questions','new_vaccinations']

# rename the columns 
df = df.rename(columns = dict(zip(df.columns,col_names)))

In [ ]:
df.drop(columns=['numbers'])

In [ ]:
result_df = features.append(df)

In [ ]:
# lets setup our first session for eval
# we chose total cases since its a complete set AND is has high correlation with the other features (Pearson)
session_2 = setup(result_df, target = 'new_vaccinations', 
                  session_id=12, 
                  log_experiment=False, 
                  experiment_name='new_vaccinations_1')

In [ ]:
best_model = compare_models()